# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [ ]:
#!pip install PyMySQL

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [3]:
bd_name = "database-1"
username = "admin"
password = "12345678"
host = "database-1.c4ofnyeynvfh.us-east-1.rds.amazonaws.com"
port = 3306

### Conexion BD

In [1]:
import pymysql

In [4]:
db = pymysql.connect(host = host,
                    user = username,
                    password = password,
                    cursorclass = pymysql.cursors.DictCursor)

cursor = db.cursor()

### Version DB

In [5]:
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print("DB version:", version)

DB version: {'VERSION()': '8.0.20'}


### Creación de DB

In [6]:
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

delete_db = '''DROP DATABASE country_database'''
cursor.execute(delete_db)

cursor.execute(create_db)

1

In [7]:
cursor.execute('''SHOW DATABASES''')
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'}]

### Creación de tablas

In [11]:
create_table = '''
CREATE TABLE country (
id INT NOT NULL auto_increment,
name TEXT,
continent TEXT,
population INT,
gdp DOUBLE,
primary key (id)
)
'''

cursor.execute(create_table)

0

### Seleccionar la BD

In [10]:
use_db = '''USE country_database'''
cursor.execute(use_db)

0

In [12]:
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_country_database': 'country'}]

In [ ]:
"""
drop_table = '''DROP TABLE country'''
cursor.execute(drop_table)
"""

### Insertar datos

In [13]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp) values('%s', '%s', '%s', '%s')
''' % ('Spain', 'Europe', '47', '1.4')

cursor.execute(insert_data)

1

### Leer datos

In [16]:
read_data = '''SELECT * FROM country'''
cursor.execute(read_data)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.4}]

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [15]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [17]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [18]:
# Creamos el CSV
import pandas as pd

df = pd.DataFrame({
    'name': ['France', 'Japan', 'Argentina', 'Brasil'],
    'continent': ['Europe', 'Asia', 'America', 'America'],
    'population': [70, 126, 44, 209],
    'gdp': [2.7, 4.9, 0.5, 1.8]
})

df.to_csv('country_data.csv', sep=';', index=False)

In [19]:
df = pd.read_csv('country_data.csv', sep=';')
df.head()

,name,continent,population,gdp
0,France,Europe,70,2.7
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.8


In [21]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                      .format(user = username,
                             host = host,
                             pw = password,
                             db = "country_database"))

In [22]:
df.to_sql('country',
         con = engine,
         if_exists = 'append',
         index = False)

In [23]:
db.commit()

In [24]:
cursor.execute(read_data)
cursor.fetchall()

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.4},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 70,
  'gdp': 2.7},
 {'id': 3,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 126,
  'gdp': 4.9},
 {'id': 4,
  'name': 'Argentina',
  'continent': 'America',
  'population': 44,
  'gdp': 0.5},
 {'id': 5,
  'name': 'Brasil',
  'continent': 'America',
  'population': 209,
  'gdp': 1.8}]

### Cerrar la conexión cuando acabemos

In [25]:
db.close()